In [3]:
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
import re
from geopy.geocoders import Nominatim
import folium
import requests
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [10]:
CLIENT_ID = 'VGGETGO2N4VUHXF5KGDMVZAI4VIZG44GYBQMA5GU0LK45TQI' # your Foursquare ID
CLIENT_SECRET = 'JF55SBY3JHDG22XVCEDKVV2DFDYFXWNI30NNYLFWQ1SRO0M0' # your Foursquare Secret
VERSION = '20200729' # Foursquare API version
ACCESS_TOKEN = 'LXWKWKEHMHJ01IDATYPQ3AJBLNXQFEBFTCUCNPD5Z0CU4GZX'
LIMIT = 4000
radius = 30000
lat = 47.6062
lng = -122.3321

In [11]:
venues_list = []
url = 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            lat,
            lng,
            VERSION,
            radius,
            LIMIT)
results = requests.get(url).json()["response"]["groups"][0]["items"]
venues_list.append([(
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

In [12]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Latitude','Longitude','Venue Name','Venue Latitude','Venue Longitude','Category']
nearby_venues    

,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Category
0,47.6062,-122.3321,Ellenos Real Greek Yogurt,47.608848,-122.340476,Frozen Yogurt Shop
1,47.6062,-122.3321,Pike Place Market,47.609195,-122.341258,Market
2,47.6062,-122.3321,Four Seasons Seattle,47.607470,-122.339369,Hotel
3,47.6062,-122.3321,Pike Place Fish Market,47.608813,-122.340371,Fish Market
4,47.6062,-122.3321,Beecher's Handmade Cheese,47.609570,-122.341851,Cheese Shop
...,...,...,...,...,...,...
95,47.6062,-122.3321,Canlis,47.643064,-122.346754,American Restaurant
96,47.6062,-122.3321,Fremont Brewing Company,47.648974,-122.344491,Brewery
97,47.6062,-122.3321,Sea Wolf Bakery,47.651034,-122.343053,Bakery
98,47.6062,-122.3321,Schilling Cider House,47.649558,-122.349232,Bar


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

In [4]:
Toronto = BeautifulSoup(page,'lxml')

In [5]:
Toronto.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"449ccf20-e1f6-4d16-813e-c13126a76ab2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":969510799,"wgRevisionId":969510799,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Ca

In [6]:
Toronto.title.string

'List of postal codes of Canada: M - Wikipedia'

In [7]:
all_tables = Toronto.find_all("table")

In [9]:
right_table = Toronto.find('table',class_ = 'wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [34]:
A = []
B = []
C = []
for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        A.append(cells[0].find(text=True).rstrip())
        B.append(cells[1].find(text=True).rstrip())
        C.append(cells[2].find(text=True).rstrip())

In [35]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [36]:
df.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [30]:
df2 = df[df.Borough != 'Not assigned']

In [31]:
df2.reset_index(drop = True,inplace=True)

In [47]:
df2

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [44]:
df2.shape

(103, 3)

In [45]:
geospatial = pd.read_csv('Geospatial_Coordinates.csv')

In [46]:
geospatial

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [48]:
df_merged = pd.merge(df2,geospatial,on='Postal Code')
df_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [51]:
df_merged2 = df_merged[df_merged['Borough'].str.contains('Toronto')]
df_merged2.reset_index(drop=True, inplace = True)

In [52]:
df_merged2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [54]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [83]:
map_Toronto = folium.Map(location = [latitude,longitude], zoom_start = 12)
for lat, lng, borough, neighborhood in zip(df_merged2['Latitude'], df_merged2['Longitude'], df_merged2['Borough'], df_merged2['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 4,
        popup = label,
        color = 'blue',
        fill=False,
        parse_html=False).add_to(map_Toronto)

In [84]:
map_Toronto

In [87]:
CLIENT_ID = 'VGGETGO2N4VUHXF5KGDMVZAI4VIZG44GYBQMA5GU0LK45TQI' # your Foursquare ID
CLIENT_SECRET = 'JF55SBY3JHDG22XVCEDKVV2DFDYFXWNI30NNYLFWQ1SRO0M0' # your Foursquare Secret
VERSION = '20200729' # Foursquare API version
ACCESS_TOKEN = 'LXWKWKEHMHJ01IDATYPQ3AJBLNXQFEBFTCUCNPD5Z0CU4GZX'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS TOEKN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: VGGETGO2N4VUHXF5KGDMVZAI4VIZG44GYBQMA5GU0LK45TQI
CLIENT_SECRET:JF55SBY3JHDG22XVCEDKVV2DFDYFXWNI30NNYLFWQ1SRO0M0
ACCESS TOEKN:LXWKWKEHMHJ01IDATYPQ3AJBLNXQFEBFTCUCNPD5Z0CU4GZX


In [97]:
def getNearByVenues(names, latitudes, longitudes, radius = 300):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            lat,
            lng,
            VERSION,
            radius,
            LIMIT)
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Latitude','Longitude','Venue Name','Venue Latitude','Venue Longitude','Category']
    return nearby_venues


In [98]:
Toronto_venues = getNearByVenues(df_merged2['Neighborhood'],df_merged2['Latitude'],df_merged2['Longitude'],radius = 300)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [99]:
Toronto_venues

,Neighborhood,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
...,...,...,...,...,...,...,...
891,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Chick-n-Joy,43.665181,-79.321403,Fast Food Restaurant
892,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Amin Car Repair Garage,43.663544,-79.320130,Auto Workshop
893,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Ashbridge Estate,43.664691,-79.321805,Garden
894,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station


In [148]:
Toronto_venues.groupby('Neighborhood').count()

,Latitude,Longitude,Venue Name,Venue Latitude,Venue Longitude,Category
Neighborhood,,,,,,
Berczy Park,8,8,8,8,8,8
"Brockton, Parkdale Village, Exhibition Place",13,13,13,13,13,13
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",7,7,7,7,7,7
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",5,5,5,5,5,5
Central Bay Street,33,33,33,33,33,33
Christie,7,7,7,7,7,7
Church and Wellesley,52,52,52,52,52,52
"Commerce Court, Victoria Hotel",76,76,76,76,76,76
Davisville,23,23,23,23,23,23


In [149]:
unique_venues = Toronto_venues['Category'].unique()
len(unique_venues)

187

In [161]:
Toronto_dummy = pd.get_dummies(Toronto_venues[['Category']], prefix = "", prefix_sep = "")
Toronto_dummy.drop(['Neighborhood'],axis=1, inplace=True)
Toronto_dummy

,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
893,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
894,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [162]:
new = Toronto_venues['Neighborhood'].to_frame()

In [167]:
Toronto_dummy2 = pd.concat([new,Toronto_dummy],axis =1)

In [168]:
Toronto_dummy2

,Neighborhood,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
892,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
893,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
894,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [169]:
Toronto_grouped = Toronto_dummy2.groupby('Neighborhood').mean().reset_index()

In [170]:
Toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.2,0.2,0.2,0.2,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Christie,0.0,0.000000,0.0,0.0,0.0,0.0,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0,0.019231,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.019231,0.000000
7,"Commerce Court, Victoria Hotel",0.0,0.000000,0.0,0.0,0.0,0.0,0.026316,0.000000,0.013158,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000
8,Davisville,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [171]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [209]:
num_top_venues = 5
columns = ['Neighborhood']
for rank in range(num_top_venues):
    columns.append('Number ' + str(rank+1) + ' popular venue type')
popular_venues = pd.DataFrame(columns = columns)
popular_venues['Neighborhood'] = Toronto_grouped['Neighborhood']
for ind in range(Toronto_grouped.shape[0]):
    popular_venues.iloc[ind,1:] = most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
popular_venues.head()

,Neighborhood,Number 1 popular venue type,Number 2 popular venue type,Number 3 popular venue type,Number 4 popular venue type,Number 5 popular venue type
0,Berczy Park,Restaurant,Coffee Shop,Concert Hall,Liquor Store,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Gym,Sandwich Place,Café,Grocery Store,Coffee Shop
2,"Business reply mail Processing Centre, South C...",Farmers Market,Garden,Light Rail Station,Brewery,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Coffee Shop
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Deli / Bodega


In [210]:
kcluster = 4
Toronto_clustering = Toronto_grouped.drop(['Neighborhood'],axis=1)
kmeans = KMeans(n_clusters = kcluster, random_state = 0).fit(Toronto_clustering)
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1,
       1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1])

In [211]:
popular_venues.insert(0, 'Cluster_Labels', kmeans.labels_)
Toronto_merge = df_merged2
Toronto_merge = Toronto_merge.join(popular_venues.set_index('Neighborhood'), on = 'Neighborhood')
Toronto_merge

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,Number 1 popular venue type,Number 2 popular venue type,Number 3 popular venue type,Number 4 popular venue type,Number 5 popular venue type
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Gym / Fitness Center,Italian Restaurant,Breakfast Spot,History Museum,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Italian Restaurant,Portuguese Restaurant,Deli / Bodega,Sandwich Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Coffee Shop,Café,Middle Eastern Restaurant,Hotel,Clothing Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1.0,Gastropub,Coffee Shop,Cosmetics Shop,Italian Restaurant,Beer Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2.0,Park,Trail,Spa,Department Store,Electronics Store
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1.0,Restaurant,Coffee Shop,Concert Hall,Liquor Store,Beer Bar
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1.0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Deli / Bodega
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1.0,Grocery Store,Candy Store,American Restaurant,Coffee Shop,Japanese Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1.0,Coffee Shop,Steakhouse,Bar,Asian Restaurant,Seafood Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1.0,Pet Store,Bakery,Grocery Store,Pharmacy,Music Venue


In [212]:
Toronto_merge2 = Toronto_merge.dropna()
Toronto_merge2.reset_index(drop = True,inplace=True)
Toronto_merge2

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,Number 1 popular venue type,Number 2 popular venue type,Number 3 popular venue type,Number 4 popular venue type,Number 5 popular venue type
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Gym / Fitness Center,Italian Restaurant,Breakfast Spot,History Museum,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Italian Restaurant,Portuguese Restaurant,Deli / Bodega,Sandwich Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Coffee Shop,Café,Middle Eastern Restaurant,Hotel,Clothing Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1.0,Gastropub,Coffee Shop,Cosmetics Shop,Italian Restaurant,Beer Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2.0,Park,Trail,Spa,Department Store,Electronics Store
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1.0,Restaurant,Coffee Shop,Concert Hall,Liquor Store,Beer Bar
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1.0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Deli / Bodega
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1.0,Grocery Store,Candy Store,American Restaurant,Coffee Shop,Japanese Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1.0,Coffee Shop,Steakhouse,Bar,Asian Restaurant,Seafood Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1.0,Pet Store,Bakery,Grocery Store,Pharmacy,Music Venue


In [217]:
map_clusters = folium.Map(location=[latitude+0.02,longitude], zoom_start = 12)
x = np.arange(kcluster)
ys = [i + x + (i*x)**2 for i in range(kcluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
marker_colors = []
for lat,lng,Neighborhood,cluster in zip(Toronto_merge2['Latitude'],Toronto_merge2['Longitude'],Toronto_merge2['Neighborhood'],Toronto_merge2['Cluster_Labels']):
    label = folium.Popup(str(Neighborhood)+' Cluster '+str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 4,
        popup = label,
        color = rainbow[int(cluster) - 1],
        fill = False).add_to(map_clusters)
map_clusters

In [218]:
Toronto_merge.loc[Toronto_merge['Cluster_Labels'] == 0.0, Toronto_merge.columns[[2] + list(range(7, Toronto_merge.shape[1]))]]

,Neighborhood,Number 2 popular venue type,Number 3 popular venue type,Number 4 popular venue type,Number 5 popular venue type
18,Lawrence Park,Photography Studio,Lawyer,Dessert Shop,Ethiopian Restaurant
21,"Forest Hill North & West, Forest Hill Road Park",Lawyer,Greek Restaurant,Gourmet Shop,Ethiopian Restaurant


In [219]:
Toronto_merge.loc[Toronto_merge['Cluster_Labels'] == 1.0, Toronto_merge.columns[[2] + list(range(7, Toronto_merge.shape[1]))]]

,Neighborhood,Number 2 popular venue type,Number 3 popular venue type,Number 4 popular venue type,Number 5 popular venue type
0,"Regent Park, Harbourfront",Italian Restaurant,Breakfast Spot,History Museum,Spa
1,"Queen's Park, Ontario Provincial Government",Italian Restaurant,Portuguese Restaurant,Deli / Bodega,Sandwich Place
2,"Garden District, Ryerson",Café,Middle Eastern Restaurant,Hotel,Clothing Store
3,St. James Town,Coffee Shop,Cosmetics Shop,Italian Restaurant,Beer Bar
5,Berczy Park,Coffee Shop,Concert Hall,Liquor Store,Beer Bar
6,Central Bay Street,Café,Italian Restaurant,Sandwich Place,Deli / Bodega
7,Christie,Candy Store,American Restaurant,Coffee Shop,Japanese Restaurant
8,"Richmond, Adelaide, King",Steakhouse,Bar,Asian Restaurant,Seafood Restaurant
9,"Dufferin, Dovercourt Village",Bakery,Grocery Store,Pharmacy,Music Venue
10,"Harbourfront East, Union Station, Toronto Islands",Café,Boat or Ferry,Hotel,Bank


In [220]:
Toronto_merge.loc[Toronto_merge['Cluster_Labels'] == 2.0, Toronto_merge.columns[[2] + list(range(7, Toronto_merge.shape[1]))]]

,Neighborhood,Number 2 popular venue type,Number 3 popular venue type,Number 4 popular venue type,Number 5 popular venue type
4,The Beaches,Trail,Spa,Department Store,Electronics Store
29,"Moore Park, Summerhill East",Yoga Studio,Design Studio,Ethiopian Restaurant,Electronics Store


In [221]:
Toronto_merge.loc[Toronto_merge['Cluster_Labels'] == 3.0, Toronto_merge.columns[[2] + list(range(7, Toronto_merge.shape[1]))]]

,Neighborhood,Number 2 popular venue type,Number 3 popular venue type,Number 4 popular venue type,Number 5 popular venue type
19,Roselawn,Dessert Shop,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
